In [1]:
import collections
import os
import random
import time
from tqdm import tqdm
import jittor as jt
from jittor import nn
from jittor.dataset.dataset import Dataset

[i 0530 16:12:03.472962 88 compiler.py:847] Jittor(1.2.2.28) src: /usr/local/lib/python3.7/dist-packages/jittor
[i 0530 16:12:03.473845 88 compiler.py:848] g++ at /usr/bin/g++
[i 0530 16:12:03.474609 88 compiler.py:849] cache_path: /root/.cache/jittor/default/g++
[i 0530 16:12:03.502645 88 __init__.py:257] Found addr2line(2.30) at /usr/bin/addr2line.
[i 0530 16:12:03.566953 88 compiler.py:889] pybind_include: -I/usr/include/python3.7m -I/usr/local/lib/python3.7/dist-packages/pybind11/include
[i 0530 16:12:03.587425 88 compiler.py:891] extension_suffix: .cpython-37m-x86_64-linux-gnu.so
[i 0530 16:12:03.786500 88 __init__.py:169] Total mem: 7.53GB, using 2 procs for compiling.
[i 0530 16:12:03.920968 88 jit_compiler.cc:21] Load cc_path: /usr/bin/g++
[i 0530 16:12:04.042809 88 __init__.py:257] Found mpicc(2.1.1) at /usr/bin/mpicc.
[i 0530 16:12:04.066444 88 compiler.py:654] handle pyjt_include/usr/local/lib/python3.7/dist-packages/jittor/extern/mpi/inc/mpi_warper.h


In [2]:
def decode_file(file_ID_name, file_result_name):
    file_ID = open(file_ID_name, "r")
    file_result = open(file_result_name, "r")
    data = []
    
    for line1, line2 in zip(file_ID, file_result):
        feature = line1.replace('\n','').lower().split(' ')
        label = line2.replace('\n','').lower().split(',')
        label = [int(x) for x in label]
        data.append([feature, label[1:]])
    
    random.shuffle(data)
    file_ID.close()
    file_result.close()
    return data

In [3]:
folder_name = "/.cached/data/"
train_ID_name = folder_name + "ID_train"
train_result_name = folder_name + "ISEAR_train"
train_data = decode_file(train_ID_name, train_result_name)

In [4]:
class MyVocab:
    def __init__(self):
        self._vocab = {}
        self._size = 1
        
    def insert(self, word):
        if word not in self._vocab.keys():
            self._vocab[word] = self._size
            self._size = self._size + 1
    
    def locate(self, word):
        if word not in self._vocab.keys():
            return 0
        return self._vocab[word]
    
    def size(self):
        return self._size

def build_vocab(data):
    vocab = MyVocab()
    for sentence in data:
        for word in sentence[0]:
            vocab.insert(word)
    return vocab

def resize_sentence(data, normal_len):
    def pad(sentence, size):
        return sentence[:size] if len(sentence) > size else sentence+[0]*(size-len(sentence))
    return [[pad(sentence[0], normal_len), sentence[1]] for sentence in data]
    

train_vocab = build_vocab(train_data)

In [5]:
class TrainDataset(Dataset):
    def __init__(self, vocab=None, data=None, normal_len=180, batch_size=1, shuffle=False):
        super().__init__()
        
        self._batch_size = batch_size
        self._shuffle = shuffle
        self._total_len = len(data)
        self._sentence_len = normal_len
        self._vocab = vocab
        self._data = []
        for sentence in data:
            self._data.append([[vocab.locate(x) for x in sentence[0]], sentence[1]])
        self._data = resize_sentence(self._data, normal_len)
        self.set_attrs(batch_size=self._batch_size, total_len=self._total_len, shuffle=self._shuffle)
            
    def __getitem__(self, index):
        feature = jt.array(self._data[index][0])
        label = jt.array(self._data[index][1])
        return feature, label
    
trainDataset = TrainDataset(vocab = train_vocab, data = train_data)

In [6]:
class Maxpool1D(jt.Module):
    def __init__(self):
        super(Maxpool1D, self).__init__()
    
    def execute(self, x):
        return jt.argmax(x, -1, False)

In [27]:
class TextCNN(jt.Module):
    def __init__(self, vocab_size, embed_size, kernel_sizes, num_channels, num_emotion = 7):
        super(TextCNN, self).__init__()
        self._embedding = nn.Embedding(vocab_size, embed_size)
        self._constant_embedding = nn.Embedding(vocab_size, embed_size)
        self._pool = Maxpool1D()
        self._convs = nn.Sequential()
        for k, c in zip(kernel_sizes, num_channels):
            self._convs.append(nn.Conv1d(in_channels = 2 * embed_size,
                                       out_channels = c,
                                       kernel_size = k,))
        self._decoder = nn.Linear(sum(num_channels), num_emotion)
        self._dropout = nn.Dropout(0.5)
        
    def execute(self, inputs):
        '''
        @params:
            inputs: 词语下标序列，形状为 (batch_size, len_sentence) 的整数张量
        @return:
            outputs: 对文本情感的预测，形状为 (batch_size, num_emotion) 的张量
        '''
        print(inputs.shape[0])
        embeddings = jt.contrib.concat([self._embedding(inputs),
                                       self._constant_embedding(inputs)], dim = -1)
        # embeddings -> (batch_size, len_sentence, embed_size*2)
        embbedings = jt.permute(0, 2, 1)
        
        encoding = jt.contrib.concat([
            self._pool(nn.relu(conv(embeddings))) for conv in self._convs], dim = -1)
        outputs = self._decoder(self._dropout(encoding))
        return outputs
    
vocab_size = train_vocab.size()
embed_size, kernel_sizes, num_channels = 100, [3, 4, 5], [100, 100, 100]
net = TextCNN(vocab_size, embed_size, kernel_sizes, num_channels)

In [28]:
# lr scheduler
def poly_lr_scheduler(opt, init_lr, index, epoch, max_iter, max_epoch):
    new_lr = init_lr * (1 - float(epoch * max_iter + index) / (max_epoch * max_iter)) ** 0.9
    opt.lr = new_lr

def train(model, train_loader, optimizer, init_lr, epoch, max_epoch):
    model.train()
    max_iter = len(train_loader)
    
    for index, (feature, label) in enumerate(train_loader):
        poly_lr_scheduler(optimizer, init_lr, index, epoch, max_iter, max_epoch)
        pred = model(feature)
        loss = nn.cross_entropy_loss(pred, label) #ignore_index?
        optimizer.step(loss)
        print ('Training in epoch {} iteration {} loss = {}'.format(epoch, index, loss.data[0]))

In [29]:
learning_rate = 0.005
momentum = 0.9
weight_decay = 1e-4
optimizer = nn.SGD(net.parameters(), learning_rate, momentum, weight_decay)
for epoch in range(max_epoch):
    train(net, trainDataset, optimizer, init_lr, epoch, max_epoch)

1


RuntimeError: Wrong inputs arguments, Please refer to examples(help(jt.ops.reshape)).

Types of your inputs are:
 self	= module,
 args	= (Var, list, ),

The function declarations are:
 VarHolder* reshape(VarHolder* x,  NanoVector shape)

Failed reason:[38;5;1m[f 0530 16:49:22.291712 88 reshape_op.cc:46] Check failed x_items(18000) == y_items(100) reshape shape is invalid for input of size[m